# LCU Demo (Linear Combination of Unitaries)

This notebook demonstrates a minimal linear combination of unitaries (LCU) workflow using QuBlock's semantic execution.

We combine two 2x2 unitaries A and B with non-negative coefficients a and b. The LCU block encoding
represents the operator a*A + b*B with normalization alpha = a + b. The post-selected LCU action
on the system should match (a*A + b*B) / alpha.

In [9]:
import numpy as np

from blockflow import (
    ApplyBlockEncodingStep,
    BlockEncoding,
    NumpyMatrixOperator,
    Program,
    ResourceEstimate,
    SemanticExecutor,
    StateVector,
)

# Define two unitaries (Pauli X and Z).
A = np.array([[0.0, 1.0], [1.0, 0.0]], dtype=complex)
B = np.array([[1.0, 0.0], [0.0, -1.0]], dtype=complex)

# LCU coefficients (non-negative).
a = 0.6
b = 0.8
alpha = a + b

lcu_matrix = a * A + b * B
lcu_normalized = lcu_matrix / alpha

print("alpha =", alpha)
print("LCU operator (normalized):")
print(lcu_normalized)

alpha = 1.4
LCU operator (normalized):
[[ 0.57142857+0.j  0.42857143+0.j]
 [ 0.42857143+0.j -0.57142857+0.j]]


In [10]:
# Prepare an input state |psi> and compute expected output.
psi = np.array([1.0, 0.0], dtype=complex)
expected = lcu_normalized @ psi
print("Expected output:", expected)

# Build a BlockEncoding for the unnormalized LCU operator.
be = BlockEncoding(
    op=NumpyMatrixOperator(lcu_matrix),
    alpha=alpha,
    resources=ResourceEstimate(),
)

# Run a semantic program with one application of the block encoding.
program = Program([ApplyBlockEncodingStep(be)])
state = StateVector(psi)
final_state, report = SemanticExecutor().run(program, state)

# The LCU algorithm output corresponds to final_state / alpha.
lcu_output = final_state.data / be.alpha
print("LCU output:", lcu_output)
print("Match expected:", np.allclose(lcu_output, expected))
print("Uses:", report.uses, "Success prob:", report.cumulative_success_prob)

Expected output: [0.57142857+0.j 0.42857143+0.j]
LCU output: [0.57142857+0.j 0.42857143+0.j]
Match expected: True
Uses: 1 Success prob: 1.0


## Circuit compilation + Qiskit cross-check

Because a**2 + b**2 = 1 in this demo, the LCU operator a*X + b*Z is unitary.
It can be implemented as Ry(theta) Z Ry(-theta), where sin(theta)=a and cos(theta)=b.
Gate order matters: the overall unitary is applied right-to-left, so we add
Ry(-theta) then Z then Ry(theta).
We build a Circuit recipe, export it to OpenQASM, and compare Qiskit
simulation to QuBlock's semantic execution.


In [11]:
from blockflow import Capabilities, Circuit, StaticCircuitRecipe, WireSpec

theta = np.arctan2(a, b)
circ = Circuit(num_qubits=1)
circ.add("ry", [0], [-theta])
circ.add("z", [0])
circ.add("ry", [0], [theta])

recipe = StaticCircuitRecipe(WireSpec(system_qubits=1), circ)
be_unitary = BlockEncoding(
    op=NumpyMatrixOperator(lcu_matrix),
    alpha=1.0,
    resources=ResourceEstimate(),
    recipe=recipe,
    capabilities=Capabilities(supports_circuit_recipe=True),
)

compiled = be_unitary.build_circuit(optimize=False)
qasm2 = be_unitary.export_openqasm(flavor="qasm2", optimize=False)
print("Compiled gates:", compiled.gates)
print(qasm2)


Compiled gates: [Gate(name='ry', qubits=(0,), params=(np.float64(-0.6435011087932844),), controls=()), Gate(name='z', qubits=(0,), params=(), controls=()), Gate(name='ry', qubits=(0,), params=(np.float64(0.6435011087932844),), controls=())]
OPENQASM 2.0;
include "qelib1.inc";
qreg q[1];
ry(-0.6435011087932844) q[0];
z q[0];
ry(0.6435011087932844) q[0];



In [12]:
try:
    from qiskit import QuantumCircuit
    from qiskit.quantum_info import Statevector
except ImportError as exc:
    raise SystemExit("Qiskit is required here. Install with: pip install qiskit") from exc

program_unitary = Program([ApplyBlockEncodingStep(be_unitary)])
state = StateVector(psi)
final_state_unitary, _ = SemanticExecutor().run(program_unitary, state)

qc = QuantumCircuit.from_qasm_str(qasm2)
qiskit_out = Statevector(psi).evolve(qc).data

print("Semantic output:", final_state_unitary.data)
print("Qiskit output:", qiskit_out)
print("Match:", np.allclose(final_state_unitary.data, qiskit_out))


Semantic output: [0.8+0.j 0.6+0.j]
Qiskit output: [0.8+0.j 0.6+0.j]
Match: True


## Small 2-qubit LCU + controlled gate (Qiskit-feasible)

This example applies the unitary LCU on qubit 0 and then a controlled-X into qubit 1.
We compile the circuit, run it in Qiskit, and verify it matches QuBlock's semantic execution.


In [13]:
try:
    from qiskit import QuantumCircuit
    from qiskit.quantum_info import Statevector
except ImportError as exc:
    raise SystemExit("Qiskit is required here. Install with: pip install qiskit") from exc

a2 = 0.6
b2 = 0.8
theta2 = np.arctan2(a2, b2)

# Single-qubit unitary implementing a*X + b*Z with a^2+b^2=1.
U1 = np.array([[b2, a2], [a2, -b2]], dtype=complex)
I = np.eye(2, dtype=complex)
CX = np.array(
    [[1, 0, 0, 0],
     [0, 1, 0, 0],
     [0, 0, 0, 1],
     [0, 0, 1, 0]],
    dtype=complex,
)
U = CX @ np.kron(U1, I)

circ_small = Circuit(num_qubits=2)
circ_small.add("ry", [0], [-theta2])
circ_small.add("z", [0])
circ_small.add("ry", [0], [theta2])
circ_small.add("cx", [0, 1])

recipe_small = StaticCircuitRecipe(WireSpec(system_qubits=2), circ_small)
be_small = BlockEncoding(
    op=NumpyMatrixOperator(U),
    alpha=1.0,
    resources=ResourceEstimate(),
    recipe=recipe_small,
    capabilities=Capabilities(supports_circuit_recipe=True),
)

psi_small = np.array([1.0, 0.0, 0.0, 0.0], dtype=complex)
program_small = Program([ApplyBlockEncodingStep(be_small)])
final_small, _ = SemanticExecutor().run(program_small, StateVector(psi_small))

qasm2_small = be_small.export_openqasm(flavor="qasm2", optimize=False)
qc_small = QuantumCircuit.from_qasm_str(qasm2_small)
qiskit_small = Statevector(psi_small).evolve(qc_small).data

print("Semantic output:", final_small.data)
print("Qiskit output:", qiskit_small)
print("Match:", np.allclose(final_small.data, qiskit_small))


Semantic output: [0.8+0.j 0. +0.j 0. +0.j 0.6+0.j]
Qiskit output: [0.8+0.j 0. +0.j 0. +0.j 0.6+0.j]
Match: True


## Larger LCU example (semantic only)

Here we use a larger number of qubits and a custom LinearOperator that applies
a global X (bit-flip) and Z (parity phase) without constructing a huge matrix.
This is intended to be run only with QuBlock's semantic executor, not Qiskit.


In [14]:
class GlobalPauliLCUOperator:
    def __init__(self, n_qubits: int, a: float, b: float):
        self.n_qubits = n_qubits
        self.a = float(a)
        self.b = float(b)
        self._dim = 1 << n_qubits
        indices = np.arange(self._dim, dtype=np.uint64)
        self._signs = np.array(
            [-1.0 if int(i).bit_count() % 2 else 1.0 for i in indices],
            dtype=complex,
        )

    @property
    def shape(self) -> tuple[int, int]:
        return (self._dim, self._dim)

    @property
    def dtype(self):
        return np.complex128

    def apply(self, vec: np.ndarray) -> np.ndarray:
        x_part = vec[::-1]
        z_part = self._signs * vec
        return self.a * x_part + self.b * z_part

    def apply_adjoint(self, vec: np.ndarray) -> np.ndarray:
        return self.apply(vec)

    def norm_bound(self) -> float:
        return abs(self.a) + abs(self.b)

n_qubits = 16
a_large = 0.4
b_large = 0.9
alpha_large = a_large + b_large
dim = 1 << n_qubits

rng = np.random.default_rng(0)
psi_large = rng.normal(size=dim) + 1j * rng.normal(size=dim)
psi_large = psi_large / np.linalg.norm(psi_large)

op_large = GlobalPauliLCUOperator(n_qubits, a_large, b_large)
be_large = BlockEncoding(
    op=op_large,
    alpha=alpha_large,
    resources=ResourceEstimate(),
)

program_large = Program([ApplyBlockEncodingStep(be_large)])
final_large, report_large = SemanticExecutor().run(program_large, StateVector(psi_large))

lcu_output_large = final_large.data / be_large.alpha
expected_large = (a_large * psi_large[::-1] + b_large * op_large._signs * psi_large) / alpha_large

print("Dim:", dim)
print("Uses:", report_large.uses)
print("Match expected:", np.allclose(lcu_output_large, expected_large))
print("Output norm:", np.linalg.norm(lcu_output_large))


Dim: 65536
Uses: 1
Match expected: True
Output norm: 0.7593427075864421
